In [1]:
import nb_utils

In [2]:
# |export
import dataclasses, json
import os
import sys
from dataclasses import dataclass, field
from typing import Any, Dict, List, NewType, Optional, Tuple

import transformers
from transformers import MODEL_FOR_CAUSAL_LM_MAPPING, HfArgumentParser

In [3]:
# |export
MODEL_CONFIG_CLASSES = list(MODEL_FOR_CAUSAL_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


DataClassType = NewType("DataClassType", Any)

In [4]:
print(MODEL_CONFIG_CLASSES[:5])
print(MODEL_TYPES[:5])

[<class 'transformers.models.bart.configuration_bart.BartConfig'>, <class 'transformers.models.bert.configuration_bert.BertConfig'>, <class 'transformers.models.bert_generation.configuration_bert_generation.BertGenerationConfig'>, <class 'transformers.models.big_bird.configuration_big_bird.BigBirdConfig'>, <class 'transformers.models.bigbird_pegasus.configuration_bigbird_pegasus.BigBirdPegasusConfig'>]
('bart', 'bert', 'bert-generation', 'big_bird', 'bigbird_pegasus')


In [5]:
# |export
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune.
    """

    base_model_revision: Optional[str] = field(
        default=None,
        metadata={"help": ("The base model checkpoint for weights initialization with PEFT adatpers.")},
    )
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={"help": ("The model checkpoint for weights initialization. Don't set if you want to train a model from scratch.")},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    model_code_revision: str = field(
        default=None,
        metadata={"help": "The branch of the IFT model"},
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype. If `auto` is passed, the "
                "dtype will be automatically derived from the model's weights."
            ),
            "choices": ["auto", "bfloat16", "float16", "float32"],
        },
    )
    trust_remote_code: bool = field(
        default=False,
        metadata={"help": "Trust remote code when loading a model."},
    )
    use_flash_attention_2: bool = field(
        default=False,
        metadata={
            "help": (
                "Whether to use flash attention 2. You must install this manually by running `pip install flash-attn --no-build-isolation`"
            )
        },
    )
    use_peft: bool = field(
        default=False,
        metadata={"help": ("Whether to use PEFT or not for training.")},
    )
    lora_r: Optional[int] = field(
        default=16,
        metadata={"help": ("LoRA R value.")},
    )
    lora_alpha: Optional[int] = field(
        default=32,
        metadata={"help": ("LoRA alpha.")},
    )
    lora_dropout: Optional[float] = field(
        default=0.05,
        metadata={"help": ("LoRA dropout.")},
    )
    lora_target_modules: Optional[List[str]] = field(
        default=None,
        metadata={"help": ("LoRA target modules.")},
    )
    lora_modules_to_save: Optional[List[str]] = field(
        default=None,
        metadata={"help": ("Model layers to unfreeze & train")},
    )
    load_in_8bit: bool = field(
        default=False,
        metadata={"help": "use 8 bit precision"},
    )
    load_in_4bit: bool = field(
        default=False,
        metadata={"help": "use 4 bit precision"},
    )

    bnb_4bit_quant_type: Optional[str] = field(
        default="nf4",
        metadata={"help": "precise the quantization type (fp4 or nf4)"},
    )
    use_bnb_nested_quant: bool = field(
        default=False,
        metadata={"help": "use nested quantization"},
    )

    def __post_init__(self):
        if self.load_in_8bit and self.load_in_4bit:
            raise ValueError("You can't use 8 bit and 4 bit precision at the same time")

In [6]:
model_args = ModelArguments(load_in_4bit=True)
model_args.__dict__

{'base_model_revision': None,
 'model_name_or_path': None,
 'model_revision': 'main',
 'model_code_revision': None,
 'torch_dtype': None,
 'trust_remote_code': False,
 'use_flash_attention_2': False,
 'use_peft': False,
 'lora_r': 16,
 'lora_alpha': 32,
 'lora_dropout': 0.05,
 'lora_target_modules': None,
 'lora_modules_to_save': None,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'bnb_4bit_quant_type': 'nf4',
 'use_bnb_nested_quant': False}

In [7]:
#|export
@dataclass
class DataArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    chat_template: Optional[str] = field(default=None, metadata={"help": "The chat template to use."})
    dataset_mixer: Optional[Dict[str, float]] = field(
        default=None,
        metadata={"help": ("Datasets and their proportions to be used for training ift/rl.")},
    )
    dataset_splits: Optional[List[str]] = field(
        default_factory=lambda: ["train", "test"],
        metadata={"help": ("List of train test splits to use in the dataset")},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": ("For debugging purposes or quicker training, truncate the number of training examples to this " "value if set.")
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": ("For debugging purposes or quicker training, truncate the number of evaluation examples to this " "value if set.")
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    truncation_side: Optional[str] = field(default=None, metadata={"help": "Truncation side to use for the tokenizer."})

In [8]:
data_args = DataArguments(chat_template="blah")
data_args.__dict__

{'chat_template': 'blah',
 'dataset_mixer': None,
 'dataset_splits': ['train', 'test'],
 'max_train_samples': None,
 'max_eval_samples': None,
 'preprocessing_num_workers': None,
 'truncation_side': None}

In [47]:
asdf = transformers.TrainingArguments("asdf")
asdf.do_eval

False

In [48]:
#|export
@dataclass
class SFTConfig(transformers.TrainingArguments):
    """
    Arguments related to the training process itself. For all parameters, see: https://huggingface.co/docs/transformers/v4.26.1/en/main_classes/trainer#transformers.TrainingArguments
    """
    max_seq_length: Optional[int] = field(
        default=None,
        metadata={"help": ("Used by TRL for reward model training, which tries to read this parameter in init.")},
    )
    logging_first_step: bool = field(
        default=True,
        metadata={"help": ("Whether to log and evaluate the first global_step or not.")},
    )
    optim: Optional[str] = field(default="adamw_torch")

In [18]:
train_args = SFTConfig(output_dir="./blah")
train_args.__dict__

{'output_dir': './blah',
 'overwrite_output_dir': False,
 'do_train': False,
 'do_eval': True,
 'do_predict': False,
 'evaluation_strategy': <IntervalStrategy.NO: 'no'>,
 'prediction_loss_only': False,
 'per_device_train_batch_size': 8,
 'per_device_eval_batch_size': 8,
 'per_gpu_train_batch_size': None,
 'per_gpu_eval_batch_size': None,
 'gradient_accumulation_steps': 1,
 'eval_accumulation_steps': None,
 'eval_delay': 0,
 'learning_rate': 5e-05,
 'weight_decay': 0.0,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'num_train_epochs': 3.0,
 'max_steps': -1,
 'lr_scheduler_type': <SchedulerType.LINEAR: 'linear'>,
 'warmup_ratio': 0.0,
 'warmup_steps': 0,
 'log_level': 'passive',
 'log_level_replica': 'warning',
 'log_on_each_node': True,
 'logging_dir': './blah/runs/Nov28_13-24-11_dl-rig',
 'logging_strategy': <IntervalStrategy.STEPS: 'steps'>,
 'logging_first_step': True,
 'logging_steps': 500,
 'logging_nan_inf_filter': True,
 'save_strategy'

In [19]:
#|export
@dataclass
class DPOConfig(transformers.TrainingArguments):
    """
    Arguments related to the DPO training process itself. For all parameters, see: https://huggingface.co/docs/transformers/v4.26.1/en/main_classes/trainer#transformers.TrainingArguments
    """

    beta: Optional[float] = field(
        default=0.1,
        metadata={"help": "The beta factor in DPO loss. Higher beta means less divergence from the initial policy."},
    )
    hub_model_revision: Optional[str] = field(
        default="main",
        metadata={"help": ("The Hub model branch to push the model to.")},
    )
    logging_first_step: bool = field(
        default=True,
        metadata={"help": ("Whether to log and evaluate the first global_step or not.")},
    )
    max_prompt_length: Optional[int] = field(
        default=None,
        metadata={"help": ("For DPO, the maximum length of the prompt to use for conditioning the model.")},
    )
    max_length: Optional[int] = field(
        default=None,
        metadata={"help": ("Used by TRL for reward model training, which tries to read this parameter in init.")},
    )
    optim: Optional[str] = field(default="rmsprop")
    remove_unused_columns: bool = field(default=False)

In [20]:
train_args = DPOConfig(output_dir="./blah")
train_args.__dict__

{'output_dir': './blah',
 'overwrite_output_dir': False,
 'do_train': False,
 'do_eval': False,
 'do_predict': False,
 'evaluation_strategy': <IntervalStrategy.NO: 'no'>,
 'prediction_loss_only': False,
 'per_device_train_batch_size': 8,
 'per_device_eval_batch_size': 8,
 'per_gpu_train_batch_size': None,
 'per_gpu_eval_batch_size': None,
 'gradient_accumulation_steps': 1,
 'eval_accumulation_steps': None,
 'eval_delay': 0,
 'learning_rate': 5e-05,
 'weight_decay': 0.0,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'num_train_epochs': 3.0,
 'max_steps': -1,
 'lr_scheduler_type': <SchedulerType.LINEAR: 'linear'>,
 'warmup_ratio': 0.0,
 'warmup_steps': 0,
 'log_level': 'passive',
 'log_level_replica': 'warning',
 'log_on_each_node': True,
 'logging_dir': './blah/runs/Nov28_13-24-12_dl-rig',
 'logging_strategy': <IntervalStrategy.STEPS: 'steps'>,
 'logging_first_step': True,
 'logging_steps': 500,
 'logging_nan_inf_filter': True,
 'save_strategy

In [49]:
# |export
class H4ArgumentParser(HfArgumentParser):
    def parse_yaml_and_args(self, yaml_arg: str, other_args: list[str] = []) -> list[dataclass]:
        """
        Parse a YAML file and overwrite the default/loaded values with the values provided to the command line.

        Args:
            yaml_arg (`str`):
                The path to the config file used
            other_args (`List[str]`, *optional`):
                A list of strings to parse as command line arguments, e.g. ['--arg=val', '--arg2=val2'].

        Returns:
            [`List[dataclass]`]: a list of dataclasses with the values from the YAML file and the command line
        """
        arg_list = self.parse_yaml_file(os.path.abspath(yaml_arg))

        outputs = []
        # strip other args list into dict of key-value pairs
        print(other_args)
        other_args = {arg.split("=")[0].strip("-"): arg.split("=")[1] for arg in other_args}
        used_args = {}

        # overwrite the default/loaded value with the value provided to the command line
        # adapted from https://github.com/huggingface/transformers/blob/d0b5002378daabf62769159add3e7d66d3f83c3b/src/transformers/hf_argparser.py#L327
        for data_yaml, data_class in zip(arg_list, self.dataclass_types):
            keys = {f.name for f in dataclasses.fields(data_yaml) if f.init}
            inputs = {k: v for k, v in vars(data_yaml).items() if k in keys}
            for arg, val in other_args.items():
                # add only if in keys
                if arg in keys:
                    base_type = data_yaml.__dataclass_fields__[arg].type
                    inputs[arg] = val

                    # cast type for ints, floats (default to strings)
                    if base_type in [int, float]:
                        inputs[arg] = base_type(val)

                    if base_type == List[str]:
                        inputs[arg] = [str(v) for v in val.split(",")]

                    # bool of a non-empty string is True, so we manually check for bools
                    if base_type == bool:
                        if val in ["true", "True"]:
                            inputs[arg] = True
                        else:
                            inputs[arg] = False
                    
                    # if dict, then load serialized json
                    try:
                        inputs[arg] = json.loads(val)
                    except:
                        pass

                    # add to used-args so we can check if double add
                    if arg not in used_args:
                        used_args[arg] = val
                    else:
                        raise ValueError(f"Duplicate argument provided: {arg}, may cause unexpected behavior")

            obj = data_class(**inputs)
            outputs.append(obj)

        return outputs

    def parse(self) -> DataClassType | Tuple[DataClassType]:
        if len(sys.argv) == 2 and sys.argv[1].endswith(".yaml"):
            # If we pass only one argument to the script and it's the path to a YAML file,
            # let's parse it to get our arguments.
            output = self.parse_yaml_file(os.path.abspath(sys.argv[1]))
        # parse command line args and yaml file
        elif len(sys.argv) > 2 and sys.argv[1].endswith(".yaml"):
            output = self.parse_yaml_and_args(os.path.abspath(sys.argv[1]), sys.argv[2:])
        # parse command line args only
        else:
            output = self.parse_args_into_dataclasses()

        if len(output) == 1:
            output = output[0]
        return output

In [58]:
parser = H4ArgumentParser((ModelArguments, DataArguments, SFTConfig))
model_args, data_args, training_args = parser.parse_yaml_and_args(yaml_arg="../configs/training_configs/zephyr-7b-beta/config_full_sft.yaml")

print(model_args)
print("=" * 80)
print(data_args)
print("=" * 80)
print(training_args)

[]
ModelArguments(base_model_revision=None, model_name_or_path='mistralai/Mistral-7B-v0.1', model_revision='main', model_code_revision=None, torch_dtype='bfloat16', trust_remote_code=False, use_flash_attention_2=True, use_peft=False, lora_r=16, lora_alpha=32, lora_dropout=0.05, lora_target_modules=None, lora_modules_to_save=None, load_in_8bit=False, load_in_4bit=False, bnb_4bit_quant_type='nf4', use_bnb_nested_quant=False)
DataArguments(chat_template=None, dataset_mixer={'HuggingFaceH4/ultrachat_200k': 1.0}, dataset_splits=['train_sft', 'test_sft'], max_train_samples=None, max_eval_samples=None, preprocessing_num_workers=12, truncation_side=None)
SFTConfig(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_paramete

In [65]:
parser = H4ArgumentParser((ModelArguments, DataArguments, SFTConfig))
model_args, data_args, training_args = parser.parse_yaml_and_args(yaml_arg="../configs/training_configs/zephyr-7b-beta/config_lora_sft.yaml", other_args=['--load_in_4bit=true', '--dataset_mixer={"HuggingFaceH4/ultrachat_200k": 0.25}', '--do_eval=false', '--evaluation_strategy=no'])

model_args.__dict__
data_args.__dict__
training_args.__dict__


['--load_in_4bit=true', '--dataset_mixer={"HuggingFaceH4/ultrachat_200k": 0.25}', '--do_eval=false', '--evaluation_strategy=no']


{'output_dir': './models/zephyr-7b-sft-lora',
 'overwrite_output_dir': True,
 'do_train': False,
 'do_eval': False,
 'do_predict': False,
 'evaluation_strategy': <IntervalStrategy.NO: 'no'>,
 'prediction_loss_only': False,
 'per_device_train_batch_size': 2,
 'per_device_eval_batch_size': 4,
 'per_gpu_train_batch_size': None,
 'per_gpu_eval_batch_size': None,
 'gradient_accumulation_steps': 128,
 'eval_accumulation_steps': None,
 'eval_delay': 0,
 'learning_rate': 2e-05,
 'weight_decay': 0.0,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'num_train_epochs': 1,
 'max_steps': -1,
 'lr_scheduler_type': <SchedulerType.COSINE: 'cosine'>,
 'warmup_ratio': 0.0,
 'warmup_steps': 0,
 'log_level': 'info',
 'log_level_replica': 'warning',
 'log_on_each_node': True,
 'logging_dir': './models/zephyr-7b-sft-lora/runs/Nov28_13-53-48_dl-rig',
 'logging_strategy': <IntervalStrategy.STEPS: 'steps'>,
 'logging_first_step': True,
 'logging_steps': 5,
 'logging_na

## Export

In [61]:
from nbdev.export import nb_export

nb_export("00_configs.ipynb", lib_path="../training_lib/", name="configs")